In [1]:
import Bio
from Bio import SeqIO
import zlib
import gzip
from io import StringIO
import gffutils
import os
import glob
import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pickle
import shutil
from termcolor import colored
import io
import Levenshtein 
from sklearn.cluster import KMeans, DBSCAN, MeanShift
import numpy as np
from sklearn.manifold import TSNE
from Bio.Seq import Seq
import itertools
import torch

2021-08-15 11:09:50,042 [7536] WARNING  py.warnings:110: [JupyterRequire] C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,

2021-08-15 11:09:50,042 [7536] WARNING  py.warnings:110: [JupyterRequire] C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
D

In [35]:
%run orf_related_functions.ipynb
%run "parsing_functions + classes.ipynb"
%run "kmer_functions.ipynb"
from Bio import SeqIO
import gffutils
import matplotlib.pyplot as plt
import numpy as np
import pomegranate
import itertools
from pomegranate import HiddenMarkovModel, State, NormalDistribution

In [36]:
a = "sssdddfff"
a[1:-1]

'ssdddff'

In [50]:

def generate_emission(state_type = "regular", probabs = None):
    symbols = genereate_kmers(3) #+ ["STA", "END"]
    if probabs != None:
        return pomegranate.distributions.DiscreteDistribution(dict(zip(symbols,probabs + [0,0])))
    else:
        if state_type == "regular":
            probabs_ = [1/len(symbols) for e in symbols]
            return pomegranate.distributions.DiscreteDistribution(dict(zip(symbols,probabs_)))
        if state_type == "start":
            probabs_ = [1/len(startcodons) if e in startcodons else 0 for e in symbols]
            return pomegranate.distributions.DiscreteDistribution(dict(zip(symbols,probabs_)))
        if state_type == "stop":
            probabs_ = [1/len(stopcodons) if e in stopcodons else 0 for e in symbols]
            return pomegranate.distributions.DiscreteDistribution(dict(zip(symbols,probabs_)))
        if state_type == "model_start":
            model_start = ["AAA"]
            probabs_ = [1/len(model_start) if e in model_start else 0 for e in symbols]
            return pomegranate.distributions.DiscreteDistribution(dict(zip(symbols,probabs_)))
        if state_type == "model_end":
            model_end = ["AAA"]
            probabs_ = [1/len(model_end) if e in model_end else 0 for e in symbols]
            return pomegranate.distributions.DiscreteDistribution(dict(zip(symbols,probabs_)))
        else:
            print(state_type, "is not a valid state_type")
            return None

def init_model_only_orfs(ncd_emissions = None, cd_emissions = None, start_emissions = None, stop_emissions = None):
    model = HiddenMarkovModel()
    ncd = State(generate_emission(state_type = "regular", probabs = ncd_emissions), name = 'ncd')
    cd = State(generate_emission(state_type = "regular", probabs = cd_emissions), name = 'cd')
    start_c = State(generate_emission(state_type = "start", probabs = start_emissions), name = 'start_c')
    stop_c = State(generate_emission(state_type = "stop", probabs = stop_emissions), name = 'stop_c')
    #model.start = State(pomegranate.DiscreteDistribution({"STA":1}),name ="None-start")
    #model.start.distribution = generate_emission(state_type = "regular")
    #model.end.distribution = generate_emission(state_type = "regular")
    #model.end = State(pomegranate.DiscreteDistribution({"END":1}),name ="None-end")
    model.add_states(ncd, cd, start_c, stop_c)
    model.add_transition(model.start, ncd, 1)
    model.add_transition(ncd, model.end, 0.1)
    model.add_transition(ncd, ncd, 0.9)
    model.add_transition(ncd, start_c, 0.1)
    model.add_transition(start_c, cd, 1)
    model.add_transition(cd, cd, 0.9)
    model.add_transition(cd, stop_c, 0.1)
    model.add_transition(stop_c, ncd, 1)
    #model.plot()
    model.bake()
    return model


def init_model_fusion(ncd_emissions = None, cd_emissions = None, start_emissions = None, stop_emissions = None):
    model = HiddenMarkovModel()
    ncd = State(generate_emission(state_type = "regular", probabs = ncd_emissions), name = 'ncd')
    cd = State(generate_emission(state_type = "regular", probabs = cd_emissions), name = 'cd')
    start_c = State(generate_emission(state_type = "start", probabs = start_emissions), name = 'start_c')
    stop_c = State(generate_emission(state_type = "stop", probabs = stop_emissions), name = 'stop_c')
    #model.start = State(pomegranate.DiscreteDistribution({"STA":1}),name ="None-start")
    #model.start.distribution = generate_emission(state_type = "regular")
    #model.end.distribution = generate_emission(state_type = "regular")
    #model.end = State(pomegranate.DiscreteDistribution({"END":1}),name ="None-end")
    model.add_states(ncd, cd, start_c, stop_c)
    model.add_transition(model.start, ncd, 1)
    model.add_transition(ncd, model.end, 0.1)
    model.add_transition(ncd, ncd, 0.9)
    model.add_transition(ncd, start_c, 0.1)
    model.add_transition(start_c, cd, 1)
    model.add_transition(cd, cd, 0.9)
    model.add_transition(cd, stop_c, 0.1)
    model.add_transition(stop_c, ncd, 1)
    
    
    #
    model.add_transition(stop_c, cd, 0.1)
    #
    #model.plot()
    model.bake()
    return model

    
def predict_genes(model, string):
    #split genes into
    
    return None #model.predict(string, algorithm='viterbi')




In [38]:
def get_viruses_of_families(path_to_virs = "D://bio_inf//annotation//splice_dataset//", fams = []):
    ans = []
    for fam in fams:
        #print(path_to_virs + fam + "//*")
        ans += glob.glob(path_to_virs + fam + "//*")
    return ans
check_gff = get_viruses_of_families(fams = ["Tombusviridae", "Solemoviridae", "Rhabdoviridae"])
check_names = [extract_name_from_path(e).split("\\")[1] for e in check_gff]


In [39]:
check_fasta = [glob.glob("D://bio_inf//annotation//genbank_fasta//" + e + "*")[0] for e in check_names]


In [40]:
check_full_annotations = extract_stats(check_gff, check_fasta)

Fusion gene: [151, 1042] Tombusviridae\GCA_000848185.1_ViralProj14638_genomic
Fusion gene: [151, 2608] Tombusviridae\GCA_000848185.1_ViralProj14638_genomic
Fusion gene: [37, 2359] Tombusviridae\GCA_000848365.1_ViralProj14674_genomic
Fusion gene: [143, 1229] Tombusviridae\GCA_000848565.1_ViralProj14715_genomic
Fusion gene: [38, 647] Tombusviridae\GCA_000848725.1_ViralProj14747_genomic
Fusion gene: [38, 2216] Tombusviridae\GCA_000848725.1_ViralProj14747_genomic
Fusion gene: [138, 1155] Tombusviridae\GCA_000850165.1_ViralProj14698_genomic
Fusion gene: [138, 1161] Tombusviridae\GCA_000850165.1_ViralProj14698_genomic
Fusion gene: [2857, 4810] Tombusviridae\GCA_000850165.1_ViralProj14698_genomic
Fusion gene: [178, 1278] Tombusviridae\GCA_000850345.1_ViralProj14734_genomic
Fusion gene: [178, 1285] Tombusviridae\GCA_000850345.1_ViralProj14734_genomic
Fusion gene: [3092, 5261] Tombusviridae\GCA_000850345.1_ViralProj14734_genomic
Fusion gene: [105, 1125] Tombusviridae\GCA_000850645.1_ViralProj14

Fusion gene: [9, 954] Tombusviridae\GCA_001271115.2_ViralProj293045_genomic
Fusion gene: [62, 671] Tombusviridae\GCA_001629905.1_ViralProj318651_genomic
Fusion gene: [62, 2237] Tombusviridae\GCA_001629905.1_ViralProj318651_genomic
Fusion gene: [283, 1546] Tombusviridae\GCA_001651105.1_ViralProj322486_genomic
Fusion gene: [283, 3205] Tombusviridae\GCA_001651105.1_ViralProj322486_genomic
Fusion gene: [77, 1070] Tombusviridae\GCA_001685265.1_ViralProj328134_genomic
Fusion gene: [77, 2636] Tombusviridae\GCA_001685265.1_ViralProj328134_genomic
Fusion gene: [63, 837] Tombusviridae\GCA_001706965.1_ViralProj338306_genomic
Fusion gene: [63, 2397] Tombusviridae\GCA_001706965.1_ViralProj338306_genomic
Fusion gene: [121, 1213] Tombusviridae\GCA_001879265.1_ViralProj353261_genomic
Fusion gene: [121, 1216] Tombusviridae\GCA_001879265.1_ViralProj353261_genomic
Fusion gene: [2947, 4891] Tombusviridae\GCA_001879265.1_ViralProj353261_genomic
Fusion gene: [44, 1046] Tombusviridae\GCA_001926835.1_ViralPro

Fusion gene: [54, 840] Solemoviridae\GCA_000909295.1_ViralProj208537_genomic
Fusion gene: [184, 2047] Solemoviridae\GCA_000909295.1_ViralProj208537_genomic
Fusion gene: [3515, 5456] Solemoviridae\GCA_000909295.1_ViralProj208537_genomic
Fusion gene: [207, 2202] Solemoviridae\GCA_000910315.1_ViralProj209366_genomic
Fusion gene: [207, 2916] Solemoviridae\GCA_000910315.1_ViralProj209366_genomic
Fusion gene: [4300, 5785] Solemoviridae\GCA_000910315.1_ViralProj209366_genomic
Fusion gene: [80, 845] Solemoviridae\GCA_000927455.1_ViralProj266722_genomic
Fusion gene: [240, 2349] Solemoviridae\GCA_000927455.1_ViralProj266722_genomic
Fusion gene: [3868, 5875] Solemoviridae\GCA_000927455.1_ViralProj266722_genomic
Fusion gene: [57, 819] Solemoviridae\GCA_000929975.1_ViralProj269794_genomic
Fusion gene: [202, 2206] Solemoviridae\GCA_000929975.1_ViralProj269794_genomic
Fusion gene: [3661, 5644] Solemoviridae\GCA_000929975.1_ViralProj269794_genomic
Fusion gene: [91, 487] Solemoviridae\GCA_001020015.1_V

Fusion gene: [70, 1423] Rhabdoviridae\GCA_000905975.1_ViralProj194143_genomic
Fusion gene: [65, 1334] Rhabdoviridae\GCA_000906815.1_ViralProj194137_genomic
Fusion gene: [65, 1337] Rhabdoviridae\GCA_000906835.1_ViralProj194141_genomic
Fusion gene: [99, 1512] Rhabdoviridae\GCA_000911135.1_ViralProj224247_genomic
Fusion gene: [103, 1378] Rhabdoviridae\GCA_000912275.1_ViralProj226731_genomic
Fusion gene: [90, 1377] Rhabdoviridae\GCA_000912795.1_ViralProj224248_genomic
Fusion gene: [64, 1333] Rhabdoviridae\GCA_000923855.1_ViralProj255105_genomic
Fusion gene: [70, 1426] Rhabdoviridae\GCA_000924435.1_ViralProj263847_genomic
Fusion gene: [63, 1332] Rhabdoviridae\GCA_000924515.1_ViralProj264901_genomic
Fusion gene: [70, 1423] Rhabdoviridae\GCA_000924535.1_ViralProj264909_genomic
Fusion gene: [63, 1320] Rhabdoviridae\GCA_000924575.1_ViralProj264917_genomic
Fusion gene: [82, 1354] Rhabdoviridae\GCA_000924675.1_ViralProj264939_genomic
Fusion gene: [97, 1516] Rhabdoviridae\GCA_000924695.1_ViralProj

Fusion gene: [0, 5347] Rhabdoviridae\GCA_002815775.1_ASM281577v1_genomic
Fusion gene: [0, 2850] Rhabdoviridae\GCA_002815795.1_ASM281579v1_genomic
Fusion gene: [0, 1224] Rhabdoviridae\GCA_002815815.1_ASM281581v1_genomic
Fusion gene: [66, 1356] Rhabdoviridae\GCA_002815855.1_ASM281585v1_genomic
Fusion gene: [47, 1331] Rhabdoviridae\GCA_002815875.1_ASM281587v1_genomic
Fusion gene: [66, 1353] Rhabdoviridae\GCA_002815895.1_ASM281589v1_genomic
Fusion gene: [84, 1377] Rhabdoviridae\GCA_002815915.1_ASM281591v1_genomic
Fusion gene: [57, 1326] Rhabdoviridae\GCA_002815975.1_ASM281597v1_genomic
Fusion gene: [63, 1332] Rhabdoviridae\GCA_002815995.1_ASM281599v1_genomic
Fusion gene: [9, 1278] Rhabdoviridae\GCA_002816015.1_ASM281601v1_genomic
Fusion gene: [64, 1333] Rhabdoviridae\GCA_002816035.1_ASM281603v1_genomic
Fusion gene: [103, 1387] Rhabdoviridae\GCA_002816055.1_ASM281605v1_genomic
Fusion gene: [59, 1346] Rhabdoviridae\GCA_002816075.1_ASM281607v1_genomic
Fusion gene: [209, 1547] Rhabdoviridae\GC

Fusion gene: [70, 1426] Rhabdoviridae\GCA_900176785.1_lib01654_genomic
Fusion gene: [70, 1426] Rhabdoviridae\GCA_900176795.1_lib01566_genomic
Fusion gene: [70, 1426] Rhabdoviridae\GCA_900176815.1_lib01936_genomic
Fusion gene: [70, 1426] Rhabdoviridae\GCA_900176945.1_lib02021_genomic
Fusion gene: [70, 1423] Rhabdoviridae\GCA_900231175.1_lib01586_genomic
Fusion gene: [70, 1423] Rhabdoviridae\GCA_900231185.1_lib01585_genomic
Fusion gene: [70, 1423] Rhabdoviridae\GCA_900231195.1_lib01442_genomic
Fusion gene: [70, 1423] Rhabdoviridae\GCA_900231205.1_lib01684_genomic
Fusion gene: [70, 1423] Rhabdoviridae\GCA_900231215.1_lib01425_genomic
Fusion gene: [70, 1423] Rhabdoviridae\GCA_900231225.1_lib01681_genomic
Fusion gene: [70, 1423] Rhabdoviridae\GCA_900231235.1_lib01427_genomic
Fusion gene: [70, 1423] Rhabdoviridae\GCA_900231245.1_lib01590_genomic
Fusion gene: [70, 1423] Rhabdoviridae\GCA_900231255.1_lib01589_genomic
Fusion gene: [70, 1423] Rhabdoviridae\GCA_900231265.1_lib01680_genomic
Fusion

In [41]:
check_frames = []
for fan in check_full_annotations:
    for an in fan.annots.values():
        check_frames += computational_split_string(an)
    

In [9]:
check_frame = tombus_frames[4]
check_frame

Tombusviridae\GCA_000848365.1_ViralProj14674_genomic , frame: 1

In [10]:
seq = check_frame.content
genes = check_frame.genes
gene = genes[0]
seq[gene[0]:gene[1]]

Seq('ATGGACGCTCTTTTACCGGAGTTTCTCGCCCACAAGCTTTCGCGTTTAGTGCTC...AAA')

In [42]:
test_frame = check_frames[0]
create_train_set(test_frame)

In [96]:
bad_name = "Rhabdoviridae\GCA_002815675.1_ASM281567v1_genomic"
bad_frame = 2
test_bad_frame = ""
for frame in check_frames:
    if frame.name == bad_name and frame.frame == bad_frame:
        test_bad_frame = frame
        break
test_bad_frame

Rhabdoviridae\GCA_002815675.1_ASM281567v1_genomic , frame: 2

In [97]:
amino_bad = test_bad_frame.splice_to_amino()

In [98]:
aa, ss = test_bad_frame.genes, len(test_bad_frame.content)
aa, ss

([[1566, 2561], [5181, 6552]], 6549)

In [99]:
test_bad_frame.content[aa[0][0]:aa[0][1]][-3:]

Seq('ACA')

In [105]:
amino_bad.genes
amino_bad.content[amino_bad.genes[0][0]:amino_bad.genes[0][1]][-3:]

[Seq('GTG'), Seq('GAA'), Seq('TAA')]

In [54]:
create_train_set(test_bad_frame)

Rhabdoviridae\GCA_002815675.1_ASM281567v1_genomic


IndexError: list index out of range

In [51]:
check_amino_frames = [create_train_set(e) for e in check_frames] 

Tombusviridae\GCA_000848185.1_ViralProj14638_genomic
Tombusviridae\GCA_000848185.1_ViralProj14638_genomic
Tombusviridae\GCA_000848185.1_ViralProj14638_genomic
Tombusviridae\GCA_000848365.1_ViralProj14674_genomic
Tombusviridae\GCA_000848365.1_ViralProj14674_genomic
Tombusviridae\GCA_000848365.1_ViralProj14674_genomic
Tombusviridae\GCA_000848565.1_ViralProj14715_genomic
Tombusviridae\GCA_000848565.1_ViralProj14715_genomic
Tombusviridae\GCA_000848565.1_ViralProj14715_genomic
Tombusviridae\GCA_000848725.1_ViralProj14747_genomic
Tombusviridae\GCA_000848725.1_ViralProj14747_genomic
Tombusviridae\GCA_000848725.1_ViralProj14747_genomic
Tombusviridae\GCA_000850165.1_ViralProj14698_genomic
Tombusviridae\GCA_000850165.1_ViralProj14698_genomic
Tombusviridae\GCA_000850165.1_ViralProj14698_genomic
Tombusviridae\GCA_000850345.1_ViralProj14734_genomic
Tombusviridae\GCA_000850345.1_ViralProj14734_genomic
Tombusviridae\GCA_000850345.1_ViralProj14734_genomic
Tombusviridae\GCA_000850645.1_ViralProj14781_g

Train set ruined and it is your fault... Frame name is Tombusviridae\GCA_001028925.1_ViralProj286706_genomic , frame: 2

skipping this one
Tombusviridae\GCA_001271115.2_ViralProj293045_genomic
Tombusviridae\GCA_001271115.2_ViralProj293045_genomic
Tombusviridae\GCA_001271115.2_ViralProj293045_genomic
Tombusviridae\GCA_001629905.1_ViralProj318651_genomic
Tombusviridae\GCA_001629905.1_ViralProj318651_genomic
Tombusviridae\GCA_001629905.1_ViralProj318651_genomic
Tombusviridae\GCA_001651105.1_ViralProj322486_genomic
Tombusviridae\GCA_001651105.1_ViralProj322486_genomic
Tombusviridae\GCA_001651105.1_ViralProj322486_genomic
Tombusviridae\GCA_001685265.1_ViralProj328134_genomic
Tombusviridae\GCA_001685265.1_ViralProj328134_genomic
Tombusviridae\GCA_001685265.1_ViralProj328134_genomic
Tombusviridae\GCA_001706965.1_ViralProj338306_genomic
Tombusviridae\GCA_001706965.1_ViralProj338306_genomic
Tombusviridae\GCA_001706965.1_ViralProj338306_genomic
Tombusviridae\GCA_001879265.1_ViralProj353261_genom

Solemoviridae\GCA_002004135.1_ViralProj375011_genomic
Solemoviridae\GCA_002004135.1_ViralProj375011_genomic
Solemoviridae\GCA_002029515.1_ViralProj379130_genomic
Solemoviridae\GCA_002029515.1_ViralProj379130_genomic
Solemoviridae\GCA_002029515.1_ViralProj379130_genomic
Solemoviridae\GCA_002080275.1_ViralProj381652_genomic
Solemoviridae\GCA_002080275.1_ViralProj381652_genomic
Solemoviridae\GCA_002080275.1_ViralProj381652_genomic
Solemoviridae\GCA_002080295.1_ViralProj381651_genomic
Solemoviridae\GCA_002080295.1_ViralProj381651_genomic
Solemoviridae\GCA_002080295.1_ViralProj381651_genomic
Solemoviridae\GCA_002087205.1_ViralProj382351_genomic
Solemoviridae\GCA_002087205.1_ViralProj382351_genomic
Solemoviridae\GCA_002087205.1_ViralProj382351_genomic
Solemoviridae\GCA_002184215.1_ViralProj388490_genomic
Solemoviridae\GCA_002184215.1_ViralProj388490_genomic
Solemoviridae\GCA_002184215.1_ViralProj388490_genomic
Solemoviridae\GCA_002270665.1_ViralProj399960_genomic
Solemoviridae\GCA_002270665.

Rhabdoviridae\GCA_000895555.1_ViralProj74995_genomic
Rhabdoviridae\GCA_000895555.1_ViralProj74995_genomic
Rhabdoviridae\GCA_000896135.1_ViralProj159049_genomic
Rhabdoviridae\GCA_000896135.1_ViralProj159049_genomic
Rhabdoviridae\GCA_000896135.1_ViralProj159049_genomic
Rhabdoviridae\GCA_000899275.1_ViralProj175665_genomic
Rhabdoviridae\GCA_000899275.1_ViralProj175665_genomic
Rhabdoviridae\GCA_000899275.1_ViralProj175665_genomic
Rhabdoviridae\GCA_000899915.1_ViralProj172457_genomic
Rhabdoviridae\GCA_000899915.1_ViralProj172457_genomic
Rhabdoviridae\GCA_000899915.1_ViralProj172457_genomic
Rhabdoviridae\GCA_000904335.1_ViralProj194138_genomic
Rhabdoviridae\GCA_000904335.1_ViralProj194138_genomic
Rhabdoviridae\GCA_000904335.1_ViralProj194138_genomic
Rhabdoviridae\GCA_000904355.1_ViralProj194142_genomic
Rhabdoviridae\GCA_000904355.1_ViralProj194142_genomic
Rhabdoviridae\GCA_000904355.1_ViralProj194142_genomic
Rhabdoviridae\GCA_000905355.1_ViralProj194140_genomic
Rhabdoviridae\GCA_000905355.1_

Rhabdoviridae\GCA_001431995.1_ViralProj301024_genomic
Rhabdoviridae\GCA_001431995.1_ViralProj301024_genomic
Rhabdoviridae\GCA_001432015.1_ViralProj301028_genomic
Rhabdoviridae\GCA_001432015.1_ViralProj301028_genomic
Rhabdoviridae\GCA_001432015.1_ViralProj301028_genomic
Rhabdoviridae\GCA_001432055.1_ViralProj301021_genomic
Rhabdoviridae\GCA_001432055.1_ViralProj301021_genomic
Rhabdoviridae\GCA_001432055.1_ViralProj301021_genomic
Rhabdoviridae\GCA_001432075.2_ViralProj301025_genomic
Rhabdoviridae\GCA_001432075.2_ViralProj301025_genomic
Rhabdoviridae\GCA_001432075.2_ViralProj301025_genomic
Rhabdoviridae\GCA_001432095.1_ViralProj301029_genomic
Rhabdoviridae\GCA_001432095.1_ViralProj301029_genomic
Rhabdoviridae\GCA_001432095.1_ViralProj301029_genomic
Rhabdoviridae\GCA_001432115.1_ViralProj301018_genomic
Rhabdoviridae\GCA_001432115.1_ViralProj301018_genomic
Rhabdoviridae\GCA_001432115.1_ViralProj301018_genomic
Rhabdoviridae\GCA_001432135.1_ViralProj301022_genomic
Rhabdoviridae\GCA_001432135.

Rhabdoviridae\GCA_002146065.1_ViralProj386364_genomic
Rhabdoviridae\GCA_002146145.1_ViralProj386340_genomic
Rhabdoviridae\GCA_002146145.1_ViralProj386340_genomic
Rhabdoviridae\GCA_002146145.1_ViralProj386340_genomic
Rhabdoviridae\GCA_002146165.1_ViralProj386344_genomic
Rhabdoviridae\GCA_002146165.1_ViralProj386344_genomic
Rhabdoviridae\GCA_002146165.1_ViralProj386344_genomic
Rhabdoviridae\GCA_002146185.1_ViralProj386349_genomic
Rhabdoviridae\GCA_002146185.1_ViralProj386349_genomic
Rhabdoviridae\GCA_002146185.1_ViralProj386349_genomic
Rhabdoviridae\GCA_002146205.1_ViralProj386353_genomic
Rhabdoviridae\GCA_002146205.1_ViralProj386353_genomic
Rhabdoviridae\GCA_002146205.1_ViralProj386353_genomic
Rhabdoviridae\GCA_002146245.1_ViralProj386361_genomic
Rhabdoviridae\GCA_002146245.1_ViralProj386361_genomic
Rhabdoviridae\GCA_002146245.1_ViralProj386361_genomic
Rhabdoviridae\GCA_002146265.1_ViralProj386365_genomic
Rhabdoviridae\GCA_002146265.1_ViralProj386365_genomic
Rhabdoviridae\GCA_002146265.

IndexError: list index out of range

In [ ]:
amfr.genes

In [ ]:
amfr.content[259]

In [14]:
simple_model = init_model_only_orfs()

In [16]:
[e.name for e in simple_model.states]

['cd', 'ncd', 'start_c', 'stop_c', 'None-start', 'None-end']

In [17]:
seq = [[str(e) for e  in amfr.content]]
simple_model.fit(seq, labels = amfr.states)

{
    "class" : "HiddenMarkovModel",
    "name" : "None",
    "start" : {
        "class" : "State",
        "distribution" : null,
        "name" : "None-start",
        "weight" : 1.0
    },
    "end" : {
        "class" : "State",
        "distribution" : null,
        "name" : "None-end",
        "weight" : 1.0
    },
    "states" : [
        {
            "class" : "State",
            "distribution" : {
                "class" : "Distribution",
                "dtype" : "str",
                "name" : "DiscreteDistribution",
                "parameters" : [
                    {
                        "AAA" : 0.021582723890320576,
                        "AAT" : 0.021582723890393275,
                        "AAG" : 0.026378884754867438,
                        "AAC" : 0.019184659877574624,
                        "ATA" : 0.01518785915717561,
                        "ATT" : 0.012789762305413229,
                        "ATG" : 0.013589138869052974,
                        "ATC" :

In [19]:
[e.name for e in simple_model.states]

['cd', 'ncd', 'start_c', 'stop_c', 'None-start', 'None-end']

In [23]:
simple_model.predict(seq[0], algorithm = "viterbi")

[4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [25]:
states = [simple_model.states[e].name for e in simple_model.predict(seq[0], algorithm = "viterbi")]
for i,s in enumerate(states):
    if s == "ncd":
        print("_", end = '')
    if s == "cd":
        print("+", end = '')
    if s == "stop_c":
        print(i, end = '')
    if s == "start_c":
        print(i, end = '')

____________13++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++260____________________281++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++786__________________________813++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [278]:
check_frame.genes

[[36, 2359], [2436, 2722]]

In [20]:
print([e.name for e in simple_model.states])

['2d66e090-9d0b-4d1a-83c4-aa92f0dc64e2', '54b6642b-aff5-4d5f-8c95-17478ba2ffe3', '85f06c21-3337-42f5-acc4-f668e4258630', 'b83f8b99-06d0-4190-9a73-cc19e4e16e11', 'None-start', 'None-end']


In [6]:
from pomegranate import *
s1 = State(NormalDistribution(5, 1))
s2 = State(NormalDistribution(1, 7))
s3 = State(NormalDistribution(8, 2))
model = HiddenMarkovModel()
model.add_states(s1, s2, s3)
model.add_transition(model.start, s1, 1.0)
model.add_transition(s1, s1, 0.7)
model.add_transition(s1, s2, 0.3)
model.add_transition(s2, s2, 0.8)
model.add_transition(s2, s3, 0.2)
model.add_transition(s3, s3, 0.9)
model.add_transition(s3, model.end, 0.1)
model.bake()